## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [52]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

import pandas as pd
from nltk.corpus import stopwords
from string import punctuation

In [2]:
# Connect to database
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [4]:
# Identify table names in the database
convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Print table names in database
tables = convention_cur.fetchall()
for table in tables:
    print(table[0])

conventions


In [75]:
# Select top 5 rows to analyze format and column names
query_results = convention_cur.execute(
                            '''
                            SELECT * FROM conventions
                            LIMIT 5
                            ''')
df = pd.DataFrame(query_results.fetchall(), columns=[desc[0] for desc in query_results.description])
print(df)

        party  night           speaker  speaker_count   time  \
0  Democratic      4           Unknown              1  00:00   
1  Democratic      4         Speaker 1              1  00:33   
2  Democratic      4         Speaker 2              1  00:59   
3  Democratic      4  Kerry Washington              1  01:07   
4  Democratic      4    Bernie Sanders              1  01:18   

                                                text text_len  \
0  Skip to content The Company Careers Press Free...      127   
1  I’m here by calling the full session of the 48...       41   
2  Every four years, we come together to reaffirm...       17   
3  We fight for a more perfect union because we a...       28   
4  We must come together to defeat Donald Trump, ...       22   

                                                file  
0  www_rev_com_blog_transcripts2020-democratic-na...  
1  www_rev_com_blog_transcripts2020-democratic-na...  
2  www_rev_com_blog_transcripts2020-democratic-na...  
3  w

In [57]:
# Punctuation
punctuation = set(punctuation) # speeds up comparison

# Stopwords
sw = stopwords.words("english")

# Removes stopwords
def remove_stop(tokens):
    
    tokens = [word for word in tokens if word not in sw]
    
    return(tokens)
 
# Removes punctuation
def remove_punctuation(text, punct_set=punctuation): 
    
    return("".join([ch for ch in text if ch not in punct_set]))

# Tokenizes the test
def tokenize(text):     
    
    return text.split()

# Applies the pipeline
def prepare_pipeline(text): 
    
    text = str.lower(text)
    text = remove_punctuation(text)
    tokens = tokenize(text)
    tokens = remove_stop(tokens)
    
    return(' '.join(tokens))

In [22]:
# Initialize empty list
convention_data = []

# Select text and party from conventions table
query_results = convention_cur.execute(
                            '''
                            SELECT text, party FROM conventions
                            ''')

# For each row in query results:
for row in query_results :
    text, party = row
    # Apply text to pipeline
    tokenized_text = prepare_pipeline(text)
    # Append list to convention_data
    convention_data.append([tokenized_text, party])
    
# The first element in the sublist is the cleaned and tokenized
# text in a single string. The second element of the sublist
# is the party. 

Let's look at some random entries and see if they look right. 

In [58]:
random.choices(convention_data,k=5)

[['jon honor devotion showing returning citizens forgotten believe person made god purpose continue give americans including former inmates best chance build new life achieve american dream great american dream i’d like ask john richard say words',
  'Republican'],
 ['navy senate liaison used carry bags overseas trips', 'Democratic'],
 ['support defend', 'Republican'],
 ['son scranton claymont wilmington become one consequential vice presidents american history accolade nonetheless rest firmly behind legacy husband father grandfather grateful nation thanks vice president joseph r biden jr lifetime service behalf united states america',
  'Democratic'],
 ['care environment', 'Democratic']]

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [59]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [60]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Initialize empty dictionary
    ret_dict = {}
    
    # Split text into tokens
    tokens = text.split()
    
    # For each token:
    for word in tokens:
        # If word is found in fw, then add to dictionary with value True
        if word in fw:
            ret_dict[word] = True
    
    # Return dictionary
    return(ret_dict)

In [61]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("some people in america are citizens",feature_words)==
                     {'people':True,'america':True,"citizens":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [62]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [63]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [64]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [65]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

### My Observations

The Naive Bayes classifier performed with 50% accuracy, which is no better than flipping a coin to determine which political party the text comes from. With that being said, the classifier still gives us valuable information by analyzing he most informative features. Out of the top 25 most informative features, 23 of them come from Republicans. The most informative features out of this list from the Republican party are 'china', 'enforcement', 'destroy', 'freedoms', 'supports', and 'crime'. The most informative features out of this list from the Democratic party are 'votes' and 'climate'. 

It is interesting that the majority of the most informative features belong to the Republican party. Though this is helpful for identifying both parties because if one of these features does appear, then we can be more confident that the text comes from a Republican speaker, and if these features do NOT appear, then we can be more confident that the text comes from a Democratic speaker.

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [66]:
# Connect to database
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [67]:
# Query results
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [68]:
# Initialize empty list
tweet_data = []

# For each row in query results:
for row in results :
    candidate, party, tweet_text = row
    # Apply text to pipeline
    text = str(tweet_text)
    tokenized_text = prepare_pipeline(text)
    # Append list to tweet_data
    tweet_data.append([tokenized_text, party])

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [69]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [70]:
# Estimate party for tweet_data_sample
for tweet, party in tweet_data_sample :
    # First find features
    tweet_features = conv_features(tweet, feature_words)
    # Apply classifer
    estimated_party = classifier.classify(tweet_features)
    
    # Print estimated party samples
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: bearlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: bgo tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: bapparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: bwexe2x80x99re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives linennhttpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: bletxe2x80x99s make even greater kag xf0x9fx87xbaxf0x9fx87xb8 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Re

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [71]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties:
    for p1 in parties:
        results[p][p1] = 0

num_to_score = 10000
random.shuffle(tweet_data)

# For each row in tweet_data:
for idx, row in enumerate(tweet_data):
    tweet, party = row    
    # First find features
    tweet_features = conv_features(tweet, feature_words)
    # Apply classifier
    estimated_party = classifier.classify(tweet_features)
   
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [72]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3605, 'Democratic': 673}),
             'Democratic': defaultdict(int,
                         {'Republican': 4723, 'Democratic': 1001})})

### Reflections
This Naive Bayes classifier does not do very well at estimating the political party from congressional tweets. The classifier is 3605/4278 = 84% accurate at classifying Republican tweets and 1001/5724 = 18% accurate at classifying Democratic congressional tweets, with an overall accuracy of 4606/10002 = 46%. This might have to do with the fact that a majority of the most informative features from the training data belonged to the Republican party. Because of this, the model over-classifies the party as Republican. 

To improve this model, I would suggest making sure the training dataset is balanced. Additionally, the feature_words might need to be altered. More records could be beneficial to aid in the training of the model, as the pattern of Democratic speeches could be better analyzed.